In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import PIL
from PIL import Image
import matplotlib.pyplot as plt

#import custom functions
from image_load import load_images, get_image_stats
from image_transform import pad_images, crop_images
from charts import plot_images

In [26]:
test_fire_images = load_images('wildfire_test_set',1,100)
test_nonfire_images = load_images('real_landscape_test_set',0,100)

In [7]:
train_data = []
test_data = []
data = []
fire_data = []
nofire_data = []
#image types 
smokeTypes = ['Heavy Smoke','Light Smoke']
timeOfDay = ['Night','Day','Morning']

# for pulling non-fire photos
nofire_processed = load_images('wildfire_train_set - Base Forest - Unprocessed',0,100)
nofire_data.extend(nofire_processed)

# for pulling fire photos
for smokeType in smokeTypes:
    for time in timeOfDay:
        wildfire_train_set_Processed = load_images(f'wildfire_train_set - Unprocessed/{smokeType}/{time}', 1, 100)
        fire_data.extend(wildfire_train_set_Processed)


815
815
613


In [16]:
test_data = fire_data[:10]+nofire_data[:130]
train_data = fire_data[10:]+nofire_data[130:]

In [17]:
images, labels = zip(*train_data)
length_mean, length_max, width_max, width_mean = get_image_stats(images)
#crop and pad images
images_cropped = pad_images(images, width_max, length_max)
images_final = crop_images(images_cropped, width_mean, length_mean)
#convert all imagees to RGB some are ARGB
images_final = [np.array(Image.fromarray(image).convert('RGB')) for image in images_final]

#add labels back in
data_train = list(zip(images_final, labels))

images, labels = zip(*test_data)
length_mean, length_max, width_max, width_mean = get_image_stats(images)
#crop and pad images
images_cropped = pad_images(images, width_max, length_max)
images_final = crop_images(images_cropped, width_mean, length_mean)
#convert all imagees to RGB some are ARGB
images_final = [np.array(Image.fromarray(image).convert('RGB')) for image in images_final]

#add labels back in
data_test = list(zip(images_final, labels))

1288
140


In [ ]:
dir = os.getcwd()
output_file1 = 'train_images_unrealEvenish_pkl.pkl'
output_file2 = 'test_images_unrealUneven_pkl.pkl'

with open('{}/artifacts/{}'.format(dir,output_file1), 'wb+') as out:
    pickle.dump(data_train, out)

with open('{}/artifacts/{}'.format(dir,output_file2), 'wb+') as out:
     pickle.dump(data_test, out)

In [48]:
test_fire_images = load_images('wildfire_test_set',1,100)
test_nonfire_images = load_images('real_landscape_test_set',0,100)

In [49]:
real_test_data = []
real_test_data.extend(test_fire_images)
real_test_data.extend(test_nonfire_images)

images, labels = zip(*real_test_data)
length_mean, length_max, width_max, width_mean = get_image_stats(images)
#crop and pad images
images_cropped = pad_images(images, width_max, length_max)
images_final = crop_images(images_cropped, width_mean, length_mean)
#convert all imagees to RGB some are ARGB
images_final = [np.array(Image.fromarray(image).convert('RGB')) for image in images_final]

# #add labels back in
real_test_data = list(zip(images_final, labels))

In [51]:
dir = os.getcwd()
output_file1 = 'real_images_test.pkl'

with open('{}/artifacts/{}'.format(dir,output_file1), 'wb+') as out:
    pickle.dump(real_test_data, out)